In [1]:
# Params
input_size = (300,300)
batch_size = 80
num_workers = 10
num_classes = 2
learning_rate = 1e-03
SGD_momentum = 0.9
epochs = 10

# Training an image classifier

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


## 1. Loading and normalizing MH dataset

In [3]:
from dataset_MH import *

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [4]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb1 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='train', input_size=input_size, augmentation=True)
mhdb2 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='train', input_size=input_size, augmentation=True)
mhdb3 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='train', input_size=input_size, augmentation=True)
mhdb4 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='train', input_size=input_size, augmentation=True)

mhdb5 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='val', input_size=input_size, augmentation=True)
mhdb6 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='val', input_size=input_size, augmentation=True)
mhdb7 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='val', input_size=input_size, augmentation=True)
mhdb8 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='val', input_size=input_size, augmentation=True)

mhdb9 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='test', input_size=input_size, augmentation=True)
mhdb10 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='test', input_size=input_size, augmentation=True)
mhdb11 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='test', input_size=input_size, augmentation=True)
mhdb12 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='test', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb1 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='train', input_size=input_size, augmentation=True)
add_mhdb2 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='train', input_size=input_size, augmentation=True)
add_mhdb3 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='train', input_size=input_size, augmentation=True)
add_mhdb4 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='train', input_size=input_size, augmentation=True)

add_mhdb5 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='val', input_size=input_size, augmentation=True)
add_mhdb6 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='val', input_size=input_size, augmentation=True)
add_mhdb7 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='val', input_size=input_size, augmentation=True)
add_mhdb8 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='val', input_size=input_size, augmentation=True)

add_mhdb9 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='test', input_size=input_size, augmentation=True)
add_mhdb10 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='test', input_size=input_size, augmentation=True)
add_mhdb11 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='test', input_size=input_size, augmentation=True)
add_mhdb12 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='test', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb1 = torch.utils.data.ConcatDataset([mhdb1, add_mhdb1])
concat_mhdb2 = torch.utils.data.ConcatDataset([mhdb2, add_mhdb2])
concat_mhdb3 = torch.utils.data.ConcatDataset([mhdb3, add_mhdb3])
concat_mhdb4 = torch.utils.data.ConcatDataset([mhdb4, add_mhdb4])

concat_mhdb5 = torch.utils.data.ConcatDataset([mhdb5, add_mhdb5])
concat_mhdb6 = torch.utils.data.ConcatDataset([mhdb6, add_mhdb6])
concat_mhdb7 = torch.utils.data.ConcatDataset([mhdb7, add_mhdb7])
concat_mhdb8 = torch.utils.data.ConcatDataset([mhdb8, add_mhdb8])

concat_mhdb9 = torch.utils.data.ConcatDataset([mhdb9, add_mhdb9])
concat_mhdb10 = torch.utils.data.ConcatDataset([mhdb10, add_mhdb10])
concat_mhdb11 = torch.utils.data.ConcatDataset([mhdb11, add_mhdb11])
concat_mhdb12 = torch.utils.data.ConcatDataset([mhdb12, add_mhdb12])


print("Train size:", concat_mhdb1.__getitem__(0)[0].size(), concat_mhdb1.__len__())
print("Val size:", concat_mhdb5.__getitem__(0)[0].size(), concat_mhdb5.__len__())
print("Test size:", concat_mhdb9.__getitem__(0)[0].size(), concat_mhdb9.__len__())

Train size: torch.Size([3, 300, 300]) 2545
Val size: torch.Size([3, 300, 300]) 424
Test size: torch.Size([3, 300, 300]) 1273


In [5]:
mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb1, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb2, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb3, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb4, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

val_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb5, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb6, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb7, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb8, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

test_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb9, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)
test_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb10, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)
test_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb11, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)
test_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb12, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

## 2. Define a GoogLeNet

In [6]:
from googlenet import *

net = GoogLeNet(in_channel=3, num_classes=num_classes, aux_block=True).cuda()

#for i in net.named_children():
#    print(i)

## 3. Define a Loss function and optimizer

In [7]:
from torch.optim.lr_scheduler import StepLR

#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
#criterion = torch.nn.BCELoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=SGD_momentum)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=8, gamma=0.96)

## 4. Train the network

In [8]:
times = 1
aux_loss_weight = 0.3

for epoch in range(epochs):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(mhdb_batch1):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda().long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        aux_1, aux_2, outputs = net(inputs) #tuple: (aux1, aux2, input)
        aux_1 = aux_1.reshape(batch_size, num_classes)
        aux_2 = aux_2.reshape(batch_size, num_classes)
        outputs = outputs.reshape(batch_size, num_classes)
        
        loss1 = criterion(aux_1, labels)
        loss2 = criterion(aux_2, labels)
        loss3 = criterion(outputs, labels)
        
        loss = aux_loss_weight*loss1 + aux_loss_weight*loss2 + loss3
        loss.backward()
        ##scheduler.step()
        optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % times == 0:
        print('[%d] loss: %.3f' %(epoch, running_loss / times))
        running_loss = 0.0
        #torch.save(model,'./data/cifar_model.pkl')

print('\nFinished Training')

[0] loss: 32.236
[1] loss: 31.046
[2] loss: 31.289
[3] loss: 27.836
[4] loss: 26.775
[5] loss: 28.252
[6] loss: 28.967
[7] loss: 26.351
[8] loss: 25.803
[9] loss: 25.177

Finished Training


## 5. Test the network on the test data

In [9]:
correct = 0
total = 0

with torch.no_grad():
    net.eval()
    for data in test_mhdb_batch1:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda().long()
        
        outputs = net(images).reshape(batch_size, num_classes)
                
        _, predicted = torch.max(outputs, dim=1)
        
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' %(1240, 100 * correct / total)) #more than 50 percent

Accuracy of the network on the 1240 test images: 63 %


In [10]:
correct1 = 0
correct2 = 0
correct3 = 0
total = 0

with torch.no_grad():
    net.train()
    for data in test_mhdb_batch1:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda().long()
        
        aux_1, aux_2, outputs = net(images) #tuple: (aux1, aux2, input)
        aux_1 = aux_1.reshape(batch_size, num_classes)
        aux_2 = aux_2.reshape(batch_size, num_classes)
        outputs = outputs.reshape(batch_size, num_classes)
        
        _, predicted1 = torch.max(aux_1, dim=1)
        _, predicted2 = torch.max(aux_2, dim=1)
        _, predicted3 = torch.max(outputs, dim=1)
        
        total += labels.size(0)
        correct1 += (predicted1 == labels).sum().item()
        correct2 += (predicted2 == labels).sum().item()
        correct3 += (predicted3 == labels).sum().item()

print('Accuracy_1 of the network on the %d test images: %d %%' %(1240, 100 * correct1 / total))
print('Accuracy_2 of the network on the %d test images: %d %%' %(1240, 100 * correct2 / total))
print('Accuracy_3 of the network on the %d test images: %d %%' %(1240, 100 * correct3 / total))

Accuracy_1 of the network on the 1240 test images: 87 %
Accuracy_2 of the network on the 1240 test images: 87 %
Accuracy_3 of the network on the 1240 test images: 63 %
